In [8]:
from pymongo import MongoClient
import datetime

## Criando uma conexão
conexao = MongoClient("mongodb://localhost:27017/")

banco = conexao["projectdb"]
coll_projects = banco["projects"]
coll_colaborators = banco["collaborators"]
coll_squads = banco["squads"]

In [15]:
# 5 
def	atualizar_status(nome_projeto, data_conclusao):
    try:
        data_conclusao = datetime.strptime(data_conclusao, "%Y-%m-%d %H:%M:%S")
    except Exception as e:
        print(f"{e} - Não foi possivel fazer a converter a data de conclusão.")
        return

    projeto = coll_projects.find({"nome": nome_projeto})
    projeto_status =  coll_projects.find({"nome": nome_projeto, "status": { "$regex": "/Conclu[í|i]do/i"}})
    data_inicio = datetime.strptime(projeto["data_inicio"], "%Y-%m-%d")
        
    if not projeto:
        print("Projeto não encontrado.")
        return
    
    if not projeto_status:
        print("O projeto não está no estado concluido")    
        return
    
    if data_conclusao < data_inicio:
        print("A data de conclusão deve ser maior que a data de início")
        return
    
    retorno = coll_projects.update_one(
        {{"nome": nome_projeto}}, {"$set": {"data_vencimento":  data_conclusao}}    
    )
    
    if retorno.modified_count == 0:
        print("Ocorreu algum erro na modificação.")
    else:
        print(f"Atualização de status ocorrida com sucesso.")